<a href="https://colab.research.google.com/github/kvinne-anc/India-Rainfall-and-Commodities-Project/blob/main/India_Rainfall_DataGeoPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install category_encoders==2.*
!pip install pandas-profiling==2.*


     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 266kB 8.7MB/s 
     |████████████████████████████████| 604kB 14.7MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 3.1MB 24.1MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 296kB 45.7MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=619b7f2390c7406728ab15c955c2c6b3f9f3f5723c0ef3f63a11dec03d76ef5c
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=8062c3bf61c5a5ef5fdb3749a6d4a09f38b5c349159d48d009b9ef40c4cb7973
  Stored in directory: /root/.cache/pip/wheels/07/1c/dc/6831446f09feb8cc199ec73a0f2f0703253f6ae013a22f4be9
Successfully built htmlmin image

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 6 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 am

In [ ]:
import zipfile
import shutil
from glob import glob
import os
import pandas as pd
import geopandas as gdp
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
!pip install PyShp
!pip install shapely

     |████████████████████████████████| 225kB 5.0MB/s 
  Created wheel for PyShp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=9a37a4faea9a4f933d10fb98b56bb05857cc97884b121b09c42deb7992c6b072
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built PyShp


In [ ]:
import fiona

In [ ]:
from zipfile import ZipFile


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import sys
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*


In [ ]:
!unzip /content/commodity_trade_statistics_data.csv.zip

Archive:  /content/commodity_trade_statistics_data.csv.zip
replace commodity_trade_statistics_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/IndiaTemps.zip

Archive:  /content/IndiaTemps.zip
replace temperatures.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/IndiaRain.zip

Archive:  /content/IndiaRain.zip
replace district wise rainfall normal.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
trade = pd.read_csv('/content/commodity_trade_statistics_data.csv')

In [ ]:
trade.head(5)

In [ ]:
trade_c = trade[['country_or_area','year','comm_code', 'flow', 'category' ,'trade_usd']]

In [ ]:
India_ex=trade[trade['flow']=='Export']

In [ ]:
India_ex.head(5)

In [ ]:
India_ex.dtypes

In [ ]:
India_ex.isnull().sum()

In [ ]:
India = India_ex.drop('flow', axis=1)
India

In [ ]:
India['comm_code'].nunique

In [ ]:
# Get indexes where name column has value Total
indexNames = India[India['comm_code'] == 'TOTAL'].index
 
# Delete these row indexes from dataFrame
India.drop(indexNames, inplace=True)

India

In [ ]:
India_x = India.groupby('commodity').trade_usd.mean().reset_index(name='trade_usd')
India_x = India.nlargest(35000,'trade_usd').reset_index()
India_x

In [ ]:
India_x.shape

In [ ]:
cat_col = India_x.columns[India_x.dtypes=='object']

(cat_col)

In [ ]:
India_x[cat_col] = India_x[cat_col].astype('category')

In [ ]:
y = India_x['trade_usd']

In [ ]:
x = India_x[['country_or_area', 'year']]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
x_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
#import category_encoders as ce
#from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler

#pipeline = make_pipeline(
 #   ce.OrdinalEncoder(),
  #  SimpleImputer(strategy='mean'), 
   # StandardScaler(), 
    #RandomForestClassifier(n_estimators=100, 
     #                          bootstrap = True,
      #                         max_features = 'sqrt')
)

In [ ]:
#pipeline.fit(x_train, y_train)

In [ ]:
my_model = LinearRegression()

In [ ]:
my_model.fit(x_train, y_train)